In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error

In [29]:
df = pd.read_csv("imdb_top1000_lae.csv")

In [31]:
df.drop(columns=["Unnamed: 0", "Series_Title", "Director", "Certificate"], inplace=True)
df["Runtime"] = df["Runtime"].str.replace(" min", "").astype(float)
df["Released_Year"] = pd.to_numeric(df["Released_Year"], errors='coerce')
df["Gross"] = df["Gross"].replace(",", "", regex=True)
df["Gross"] = pd.to_numeric(df["Gross"], errors='coerce')
df.dropna(inplace=True)

In [33]:
X = df.drop(columns=["IMDB_Rating"])
y = df["IMDB_Rating"]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
#Inicializar y entrenar los modelos
models = {
    "Lineal": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1)
}

In [39]:
param_grid = {"alpha": np.logspace(-3, 3, 10)}

ridge_search = GridSearchCV(Ridge(), param_grid, scoring="neg_mean_squared_error", cv=5)
ridge_search.fit(X_train, y_train)
best_ridge = Ridge(alpha=ridge_search.best_params_["alpha"])

lasso_search = GridSearchCV(Lasso(), param_grid, scoring="neg_mean_squared_error", cv=5)
lasso_search.fit(X_train, y_train)
best_lasso = Lasso(alpha=lasso_search.best_params_["alpha"])

C:\Users\arroy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=6.71712e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\arroy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=5.11732e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\arroy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=7.01556e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\arroy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=5.4595e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\arroy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:204

In [41]:
models["Ridge"] = best_ridge
models["Lasso"] = best_lasso

In [43]:
results = {}
predictions = {} 

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions[name] = y_pred
    mse = mean_squared_error(y_test, y_pred)
    results[name] = np.sqrt(mse)

In [45]:
best_model = min(results, key=results.get)
print(f"El mejor modelo es: {best_model} con RMSE de {results[best_model]:.4f}")

El mejor modelo es: Lasso con RMSE de 0.2656


#### El modelo Lasso obtuvo el menor error, por lo que es la mejor opción. Tiene la ventaja de reducir la complejidad del modelo al eliminar características irrelevantes.